In [3]:
# Full setup
import duckdb
import pandas as pd
from sodapy import Socrata
import database_setup.database_setup as dbs
import database_setup.mta_dataset as mta
from dotenv import load_dotenv
import os

load_dotenv()

%reload_ext sql
conn = dbs.get_database_connection(os.getenv('DEV_DATABASE'))
%sql conn --alias duckdb


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [4]:
# Open Data Constsants
NY_OPEN_DATA_API_TOKEN = os.getenv('NY_OPEN_DATA_API_TOKEN')
NY_OPEN_DATA_USERNAME = os.getenv('NY_OPEN_DATA_USERNAME')
NY_OPEN_DATA_PASSWORD = os.getenv('NY_OPEN_DATA_PASSWORD')

# Database Constants
MTA_SCHEMA = os.getenv('MTA_SCHEMA')
DEV_DATABASE = os.getenv('DEV_DATABASE')
MANUAL_ENTRY_SCHEMA = os.getenv('MANUAL_ENTRY_SCHEMA')

## Testing new filter on the Socrata Client

In [20]:
start_time = "'2023-01-02T00:00:00'"
end_time = "'2023-01-04T0:00:00'"

client = Socrata("data.ny.gov",
                     NY_OPEN_DATA_API_TOKEN,
                     username=NY_OPEN_DATA_USERNAME,
                     password=NY_OPEN_DATA_PASSWORD)

results = client.get(mta.OriginDestination.code,
                    where=f"day_of_week in ('Monday', 'Tuesday') AND month = 1 AND year = 2023",
                    limit=100000000)


records_df = pd.DataFrame.from_records(results)

## Testing large extraction

In [4]:
od_jan = mta.OriginDestination()

In [5]:
start_time = "'2023-01-02T00:00:00'"
end_time = "'2023-01-31T0:00:00'"

od_jan.default_where_clause = f"timestamp between {
    start_time} and {end_time}"

In [9]:
client = Socrata("data.ny.gov",
                     NY_OPEN_DATA_API_TOKEN,
                     username=NY_OPEN_DATA_USERNAME,
                     password=NY_OPEN_DATA_PASSWORD)

results = client.get(od_jan.code,
                    where=od_jan.default_where_clause,
                    limit=100000000)


records_df = pd.DataFrame.from_records(results)

In [13]:
records_df.size

150012640

In [14]:
records_df.to_csv('test_size.csv')

In [12]:
conn.sql('select count(*) from mta.origin_destination')

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│      2903057 │
└──────────────┘

In [1]:
def year_setup(overwrite=False) -> None:
    """

    """
    od_template = mta.OriginDestination()
    hr_template = mta.HourlyRidership()
    default_name_od = od_template.table_name
    default_name_hr = hr_template.table_name

    timestamp_dict = {
        1: {'start': "'2023-01-01T00:00:00'", 'end': "'2023-02-01T00:00:00'"},
        2: {'start': "'2023-02-01T00:00:00'", 'end': "'2023-03-01T00:00:00'"},
        3: {'start': "'2023-03-01T00:00:00'", 'end': "'2023-04-01T00:00:00'"},
        4: {'start': "'2023-04-01T00:00:00'", 'end': "'2023-05-01T00:00:00'"},
        5: {'start': "'2023-05-01T00:00:00'", 'end': "'2023-06-01T00:00:00'"},
        6: {'start': "'2023-06-01T00:00:00'", 'end': "'2023-07-01T00:00:00'"},
        7: {'start': "'2023-07-01T00:00:00'", 'end': "'2023-08-01T00:00:00'"},
        8: {'start': "'2023-08-01T00:00:00'", 'end': "'2023-09-01T00:00:00'"},
        9: {'start': "'2023-09-01T00:00:00'", 'end': "'2023-10-01T00:00:00'"},
        10: {'start': "'2023-10-01T00:00:00'", 'end': "'2023-11-01T00:00:00'"},
        11: {'start': "'2023-11-01T00:00:00'", 'end': "'2023-12-01T00:00:00'"},
        12: {'start': "'2023-12-01T00:00:00'", 'end': "'2024-01-01T00:00:00'"}
    }

    with dbs.get_database_connection(DEV_DATABASE) as conn:
        for month in range(12,13):
            print(f'Building tables for month={month}')
            where_clause = f"month = {month} AND year = 2023"
            od_monthly = mta.OriginDestination(table_name = default_name_od+f'_{month}', default_where_clause=where_clause)
            dbs.new_create_table(conn, od_monthly, overwrite=True)

            hr_where_clause = f"transit_timestamp >= {timestamp_dict[month].get('start')} and transit_timestamp < {
                timestamp_dict[month].get('end')}"

            hr_monthly = mta.HourlyRidership(
                table_name=default_name_hr+f'_{month}', default_where_clause=hr_where_clause)
            dbs.new_create_table(conn, hr_monthly)

In [5]:
year_setup()

Building tables for month=12
Starting extract for origin_destination_12
Extract successful
Creating table in database


In [6]:
conn.close()

In [15]:
conn.sql("select * from information_schema.tables where table_schema='mta'")

┌───────────────┬──────────────┬──────────────────────┬────────────┬──────────────────────────────┬──────────────────────┬───────────────────────────┬──────────────────────────┬────────────────────────┬────────────────────┬──────────┬───────────────┬───────────────┐
│ table_catalog │ table_schema │      table_name      │ table_type │ self_referencing_column_name │ reference_generation │ user_defined_type_catalog │ user_defined_type_schema │ user_defined_type_name │ is_insertable_into │ is_typed │ commit_action │ TABLE_COMMENT │
│    varchar    │   varchar    │       varchar        │  varchar   │           varchar            │       varchar        │          varchar          │         varchar          │        varchar         │      varchar       │ varchar  │    varchar    │    varchar    │
├───────────────┼──────────────┼──────────────────────┼────────────┼──────────────────────────────┼──────────────────────┼───────────────────────────┼──────────────────────────┼──────────────────────

In [18]:
conn.sql('select count(*) from mta.hourly_ridership_1')

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│      2147754 │
└──────────────┘